In [10]:
import sys
sys.path.append("../../src")
import os
import datetime
import numpy as np
from example_beam import get_data
from sindy_utils import library_size
from training import train_network
from scipy.io import savemat
import tensorflow as tf

# Generate data

In [11]:
training_data, validation_data, linear_parameters = get_data(train=r'data\trajectories\non_linear_3.mat', test=r'data\trajectories\non_linear_2.mat')

# Set up model and training parameters

In [12]:
params = {}

params['save_results'] = False
params['use_bias'] = False

# reduced space & dynamics parameters
params['input_dim'] = training_data['x'].shape[-1]
params['latent_dim'] = 1
params['model_order'] = 2
params['poly_order'] = 3
params['include_sine'] = False
params['include_constant'] = False
params['library_dim'] = library_size(params['model_order']*params['latent_dim'], params['poly_order'], params['include_sine'], params['include_constant'])

params['coefficient_initialization'] = 'constant'

# loss function weighting
params['sequential_thresholding'] = False

params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_x'] = sum(sum(training_data['x']**2))/sum(sum(training_data['ddx']**2))
params['loss_weight_sindy_z'] = 0.1 * sum(sum(training_data['x']**2))/sum(sum(training_data['ddx']**2))
params['loss_weight_sindy_regularization'] = 0

# enforcing linear terms
m = linear_parameters['m']
c = linear_parameters['c']
k = linear_parameters['k']

linear_terms = [(0, -k/m), (1, -c/m)] # List of tuples containing (index, coefficient) of the a-priori known linear model
params['linear_terms'] = linear_terms

# network architecture
params['activation'] = 'elu'
params['widths'] = [64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1000
params['learning_rate'] = 1e-4

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 15001
params['refinement_epochs'] = 0

# Run training experiments

In [ ]:
num_experiments = 10
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
    tf.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)

    save_name = 'data\learned_parameters\SPLIT_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f") + '.mat'
    savemat(save_name, results_dict['learned_parameters'])

# Generate plots of loss function
The last experiment is used to generate these plots

In [ ]:
import matplotlib.pyplot as plt

titles = ['Enc-Dec','SINDy (latent space)','SINDy (decoded)','SINDy regularization']
fig, axs = plt.subplots(2,2)
for i in range(4):
    plt.subplot(2, 2, i+1)
    plt.plot(np.arange(0,100*results_dict['training_losses'].shape[0],100),results_dict['training_losses'][:,i+1], label='Training')
    plt.plot(np.arange(0,100*results_dict['validation_losses'].shape[0],100),results_dict['validation_losses'][:,i+1], label='Testing')
    plt.yscale('log')
    plt.title(titles[i])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
plt.tight_layout()
plt.show()

plt.figure()
plt.plot(np.arange(0,100*results_dict['training_losses'].shape[0],100),results_dict['training_losses'][:,0], label='Training')
plt.plot(np.arange(0,100*results_dict['validation_losses'].shape[0],100),results_dict['validation_losses'][:,0], label='Testing')
plt.yscale('log')
plt.title('(Weighted) Total')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.tight_layout()
plt.legend()
plt.show()